<a href="https://colab.research.google.com/github/kylewtho/boardgames-project/blob/main/Quantum_tic_tac_toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up the game logic

In [ ]:
!pip install qiskit --quiet
!pip install pylatexenc --quiet

from qiskit import *
from qiskit.visualization import plot_histogram
from google.colab import widgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Button, Layout
import ipywidgets
import pandas as pd
import math
import numpy as np
import random
from IPython.display import clear_output

In [ ]:
class Board:
    def __init__(self):
        self.qc = QuantumCircuit(game_size, game_size)
        self.function = ''
        self.target = -1
        self.tab = []
        self.winsX = 0
        self.winsO = 0
        self.steps = 3 #maximum number of moves 
        print("Moves remaining: " + str(self.steps))   

        for idx in range(0, game_size):
            self.tab.append({'default':str(idx), 'player':' '})
            self.qc.reset(idx)
            self.qc.h(idx)
        self.qc.barrier()

    def make_move(self, cell):
        if self.function == 'Not':
            self.qc.x(cell)
            self.tab[int(cell)]['player'] += 'N - '  
        elif self.function == 'O':
            self.qc.ry(1*np.pi/2, cell)
            self.tab[int(cell)]['player'] += "O - "
        elif self.function == 'X':
            self.qc.ry(1*-np.pi/2, cell)
            self.tab[int(cell)]['player'] += "X - " 
        elif self.function == 'SWAP' and self.target != cell:
            if self.target == cell:
                self.target = -1
            else:
                self.qc.swap(cell, self.target)
                self.tab[int(cell)]['player'] += "S - " 
                self.tab[int(self.target)]['player'] += "S - "
        self.steps -= 1
        print("Moves remaining: " + str(self.steps))
                          
    def results(self):
        display(self.qc.draw('mpl'))
        self.qc = QuantumCircuit(game_size, game_size)

    def display(self):
        display(self.qc.draw('mpl'))
        
    def measure(self):
        self.qc.barrier()
        for i in range(0,game_size):
            self.qc.measure(i, i)
        
        job = qiskit.execute(self.qc, backend, shots=1, memory=True)
        output = job.result().get_memory()[0]
        
        for i in range(0,game_size):
            if output[game_size-1-i] == '0':
                self.tab[i]['player'] = 'X'
            else:
                self.tab[i]['player'] = 'O'
        self.winsX = self.countWinners('X')
        self.winsO = self.countWinners('O')

    def countWinners(self, player):
        if game_size == 9:
          winners = ((0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6))
        else:
          winners = ((0,1),(2,3),(0,2),(1,3),(0,3),(1,2))
        wins = 0
        for i in range(len(winners)):
            won = True
            for j in range(len(winners[0])):
                if not self.tab[winners[i][j]]['player'] == player:
                    won = False
            if won:
                wins = wins + 1
        return wins
    
    def new(self):
        self.tab.clear()
        for idx in range(0,game_size):
           self.tab.append({'default':str(idx), 'player':''})
           self.qc.reset(idx)
           self.qc.h(idx)
        self.qc.barrier()


In [ ]:
class Game:
    def __init__(self):
        self.selecting = False
        self.board = Board()
        self.boardbutton_list = []
        for i in range(0,game_size):
            button = Button(description=str(i))
            button.on_click(self.handle_game)
            self.boardbutton_list.append(button)
        self.funcbutton_list = []
        self.newButton('Measure')
        self.newButton('Not')
        self.newButton('O')
        self.newButton('X')
        self.newButton('SWAP')
        self.printmenu()
        self.printBoard()

    def newButton(self, name):
        function = Button(description=name, layout=Layout(width='86px', height='30px'))
        function.on_click(self.handle_game)
        self.funcbutton_list.append(function)

    def handle_game(self, b):
        try:
            if b.description == 'Measure':
                clear_output()
                self.replay()
                self.board.measure()
                self.scoreboard()
                self.printBoard() 
                self.board.results()
                
            if b.description == 'Replay':
                clear_output()
                self.board.new()
                self.printmenu()
                self.printBoard()
                            
            if int(b.description) >= 0:
                if self.selecting:
                    self.board.target = int(b.description)
                else:
                    clear_output()
                    self.printmenu()
                    self.board.make_move(int(b.description))
                    self.printBoard()
                    self.board.display()
            self.selecting = False
        except ValueError:
            self.board.function = b.description
            self.selecting = False
            if self.board.function == 'SWAP':
                self.selecting = True
        if self.board.steps == 0:
            clear_output()
            self.replay()
            self.board.measure()
            self.scoreboard()
            self.printBoard() 
            self.board.results()
            self.board.steps = 3 #reset maximum numbers of moves

    def printmenu(self):
        grid = widgets.Grid(1, 5)
        for (row, col) in grid:
            display(self.funcbutton_list[col])

    def scoreboard(self):
        print("X wins: " + str(self.board.winsX) + "    O wins: " + str(self.board.winsO))

    def replay(self):
        rep = Button(description="Replay")
        rep.on_click(self.handle_game)
        display(rep)

    def printBoard(self):
        grid = widgets.Grid(1, int(np.sqrt(game_size)), header_row=True, header_column=True)
        for row in range(int(np.sqrt(game_size))):
              for (useless, col) in grid:
                  print("\n"+self.board.tab[col + row * int(np.sqrt(game_size))]['player']+"\n")
                  display(self.boardbutton_list[col + row * int(np.sqrt(game_size))])

In [ ]:
backend = Aer.get_backend('aer_simulator')

game_size = 9

In [ ]:
game = Game()

,,,,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='Measure', layout=Layout(height='30px', width='86px'), style=ButtonStyle())

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='8', style=ButtonStyle())

<IPython.core.display.Javascript object>